In [25]:
import pandas as pd
import holidays

In [15]:
vendas_df = pd.read_csv('vendas_diarias_202401_202509.csv')

In [16]:
vendas_df.head()

,data,loja,quantidade de vendas
0,2024-01-01,1,80
1,2024-01-01,2,84
2,2024-01-01,3,100
3,2024-01-01,4,118
4,2024-01-01,5,108


In [17]:
vendas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3195 entries, 0 to 3194
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   data                  3195 non-null   object
 1   loja                  3195 non-null   int64 
 2   quantidade de vendas  3195 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 75.0+ KB


In [18]:
vendas_df.shape

(3195, 3)

In [21]:
vendas_df['anomesdia'] = pd.to_datetime(vendas_df['data']).dt.strftime("%Y%m%d")
vendas_df['data'] = pd.to_datetime(vendas_df['data'])

In [22]:
is_weekday = vendas_df['data'].dt.weekday < 5

# numeração do dia útil dentro de cada mês (sem feriados)
vendas_df['num_dia_util_sem_feriado'] = (
    is_weekday.astype(int)
    .groupby(vendas_df['data'].dt.to_period('M')) # agrupa por mês/ano
    .cumsum() # acumula só nos dias úteis
    .where(is_weekday) # deixa NaN em sab/dom
)

In [29]:
# pega os feriados do Brasil
anos = vendas_df['data'].dt.year.unique().tolist()
feriados_br = holidays.Brazil(years=anos)
feriado_set = set(feriados_br.keys())

is_holiday = vendas_df['data'].dt.date.isin(feriado_set)
is_business_day = is_weekday & ~is_holiday

vendas_df['num_dia_util_com_feriado'] = (
    is_business_day.astype(int)
    .groupby(vendas_df['data'].dt.to_period('M'))
    .cumsum()
    .where(is_business_day) # NaN para feriados
)

In [30]:
vendas_df.head()

,data,loja,quantidade de vendas,anomesdia,num_dia_util_sem_feriado,num_dia_util_com_feriado
0,2024-01-01,1,80,20240101,1.0,NaN
1,2024-01-01,2,84,20240101,2.0,NaN
2,2024-01-01,3,100,20240101,3.0,NaN
3,2024-01-01,4,118,20240101,4.0,NaN
4,2024-01-01,5,108,20240101,5.0,NaN
